In [1]:
import torch.nn.functional

from helper import *


HOMOGENEOUS MODEL:
 GNN in gnn.py

In [11]:
# Load the homogeneous model
homogeneous_model = load_homogeneous_supervised_model()

In [12]:
#Read the grid names from directory
grid_names = [_ for _ in os.listdir(os.path.dirname(os.path.abspath("gnn.ipynb")) + "\\data\\Supervised\\")]

# Read the homogeneous Datasets in CSV format and convert them to graph data
# Append the graph data to a list
graphdata_lst = read_multiple_supervised_datasets(grid_names)

Calculating edge index and edge attributes for the grid 1-HV-mixed--0-no_sw ...
Reading all of the .csv files from the directory of 1-HV-mixed--0-no_sw ...
Processing Training Data for 1-HV-mixed--0-no_sw ...
Processing Validation Data for 1-HV-mixed--0-no_sw ...
Processing Test Data for 1-HV-mixed--0-no_sw ...
Processing complete.
Calculating edge index and edge attributes for the grid 1-HV-urban--0-no_sw ...
Reading all of the .csv files from the directory of 1-HV-urban--0-no_sw ...
Processing Training Data for 1-HV-urban--0-no_sw ...
Processing Validation Data for 1-HV-urban--0-no_sw ...
Processing Test Data for 1-HV-urban--0-no_sw ...
Processing complete.
Calculating edge index and edge attributes for the grid 1-MV-comm--0-no_sw ...
Reading all of the .csv files from the directory of 1-MV-comm--0-no_sw ...
Processing Training Data for 1-MV-comm--0-no_sw ...
Processing Validation Data for 1-MV-comm--0-no_sw ...
Processing Test Data for 1-MV-comm--0-no_sw ...
Processing complete.
Cal

In [13]:
from torch_geometric.loader import DataLoader

# Initialize the list of test loaders
test_loaders_lst = []
# Iterate over each graph data

for graph_data in graphdata_lst:
    grid_name = graph_data.grid_name
    ln_train_dataset = len(graph_data.train_data)
    ln_val_dataset = len(graph_data.val_data)
    ln_test_dataset = len(graph_data.test_data)

    # Print the grid name and the lengths of training,validation and test datasets
    print (f"Grid Name: {grid_name}, Len_Train: {ln_train_dataset},Len_Val: {ln_val_dataset},Len_Test: {ln_test_dataset}")

    #Append the test dataset of the network graph to the list
    test_loader =  DataLoader(graph_data.test_data, batch_size=1, shuffle=True)
    test_loaders_lst.append(test_loader)

Grid Name: 1-HV-mixed--0-no_sw, Len_Train: 85,Len_Val: 10,Len_Test: 5
Grid Name: 1-HV-urban--0-no_sw, Len_Train: 85,Len_Val: 10,Len_Test: 5
Grid Name: 1-MV-comm--0-no_sw, Len_Train: 85,Len_Val: 10,Len_Test: 5
Grid Name: 1-MV-semiurb--0-no_sw, Len_Train: 85,Len_Val: 10,Len_Test: 5


In [14]:
def test_homogeneous_model(test_loader_lst, model, loss_fn):
    test_rmse_loss = 0.0
    test_mae_loss = 0.0
    test_mre_loss = 0.0
    out = []

    # create a criterion to measure the mean absolute error
    mae_loss_fn = nn.L1Loss()

    # create a criterion to measure the mean relative error (MRE), outputs, targets : Torch.Tensor
    mre_loss_fn = lambda outputs, targets: get_mre_loss(outputs, targets)

    for i, test_loader in enumerate(test_loader_lst):

        # Here, we use enumerate(validation_loader) instead of
        # iter(validation_loader) so that we can track the batch
        # index and do some intra-epoch reporting
        for j, data in enumerate(test_loader):
            scaler = StandardScaler()
            inputs, targets = data.x, data.y

            # Get edge_index and edge_attr from DataLoader
            edge_index = test_loader.dataset[j].edge_index
            edge_attr = test_loader.dataset[j].edge_attr

            # Define Scaler and standardize inputs and targets
            targets = torch.tensor(scaler.fit_transform(targets), dtype=torch.float32)
            inputs = torch.tensor(scaler.transform(inputs), dtype=torch.float32)

            # Make predictions for this batch
            outputs = model(inputs, edge_index, edge_attr=edge_attr)

            # Compute the loss and its gradients
            rmse_loss = torch.sqrt(loss_fn(outputs, targets))

            # Compute MAE loss
            mae_loss = mae_loss_fn(outputs, targets)

            # Compute MRE loss
            mre_loss = mre_loss_fn(outputs, targets)

            # Gather data and report
            test_rmse_loss += rmse_loss.item()
            test_mae_loss += mae_loss.item()
            test_mre_loss += mre_loss.item()

            if j + 1 == len(test_loader.dataset):
                output = scaler.inverse_transform(outputs.detach().numpy())
                target = scaler.inverse_transform(targets.detach().numpy())
                out.append((output, target))

    num_samples = len(test_loader_lst) * len(test_loader_lst[0].dataset)
    rmse = test_rmse_loss / num_samples
    mae = test_mae_loss / num_samples
    mre = test_mre_loss / num_samples
    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")
    print(f"MRE: {mre}")
    return out

outputs_and_targets = test_homogeneous_model(test_loaders_lst, homogeneous_model, nn.MSELoss())

RMSE: 0.002404558245325461
MAE: 0.0015261667082086205
MRE: 0.020806053886190057


HETEROGENEOUS MODELS in heterognn.py:
    --Node Type-Based Heterogeneous GNN: ACOPFGENERAL
    --Edge Type-Based Heterogeneous GNN: ACOPFGNN
    --Node Type-Based Heterogeneous GNN with Embedded Constraint Enforcement: ACOPFEmbedder_Bus_Constrained

In [15]:
# Load the node-type based heterogeneous GNN
heterogeneous_model = load_ACOPFGeneral_model('1-HV-mixed--0-no_sw', "hetero_model_bus_final.pt",16,1)

no costs are given - overall generated power is minimized
no costs are given - overall generated power is minimized


defaultdict(<class 'list'>, {'SB': tensor([[4.3778, 4.3980, 1.2443, 0.1817]], grad_fn=<ViewBackward0>), 'PQ': tensor([[ 0.6957, -0.2536, -2.1259,  0.2346],
        [-0.5310,  2.2922, -0.7527,  2.8672],
        [ 0.1181, -2.6972,  3.4727, -2.5558],
        [ 2.1422, -2.7961,  3.6309,  5.2956],
        [-0.5516, -1.4706, -0.3314, -0.6005],
        [-1.1938, -4.3016, -1.9444, -1.3011],
        [ 2.6649,  1.7119,  2.8390,  0.4738],
        [-1.3251,  1.4579, -2.1068,  1.9568],
        [ 1.4279, -0.0398, -3.1550,  1.8049],
        [-1.3081,  3.9212,  0.4466, -2.7417],
        [ 1.4596,  0.5066, -0.2251,  0.1342],
        [-0.9125, -2.2541, -0.7617, -3.4120],
        [ 0.9590,  2.5350,  1.3388, -1.1577],
        [-2.8009,  0.6519,  3.9448,  0.6603],
        [ 2.5201,  0.3159, -2.3347,  1.7825],
        [-0.2740,  2.0715, -0.2225,  0.7886]], grad_fn=<ViewBackward0>), 'PV': tensor([[ 0.4633,  0.9339, -0.3001,  0.1313],
        [-2.0569, -1.0954, -0.3790,  1.7793],
        [ 1.9897,  1.1761,  0

In [16]:
def load_test_data():
    path = "../code/data/Heterogeneous/TestData"
    with open(os.path.join(path, 'testdata.pkl'), 'rb') as f:
        test_data = pickle.load(f)

    print("Test Data Loaded.")
    return  test_data

In [17]:
# Load Test Data
test_inputs = load_test_data()

Test Data Loaded.


In [9]:
test_inputs[0]

In [21]:
def test_heterogeneous_model(model, test_inputs):
    test_rmse_loss = 0.0
    test_mae_loss = 0.0
    test_mre_loss = 0.0

    for j, ACOPFinput in enumerate(test_inputs):

        # Store the attributes in variables to be used later on
        x_dict = ACOPFinput.x_dict
        constraint_dict = ACOPFinput.constraint_dict
        edge_idx_dict = ACOPFinput.edge_idx_dict
        edge_attr_dict = ACOPFinput.edge_attr_dict
        bus_idx_neighbors_dict = ACOPFinput.bus_idx_neighbors_dict
        net = ACOPFinput.net
        scaler = ACOPFinput.scaler
        res_dict = ACOPFinput.res_bus

        out_dict = model(x_dict, bus_idx_neighbors_dict, edge_idx_dict, edge_attr_dict)

        target, output = [],[]
        for node_type in out_dict:
            output.extend(out_dict[node_type])
            target.extend(res_dict[node_type])

        x = torch.stack(output)
        y = torch.stack(target)

        rmseloss = torch.sqrt(torch.nn.functional.mse_loss(x, y))
        maeloss = torch.nn.functional.l1_loss(x,y)
        mreloss = get_mre_loss(x, y)

        # Add the losses
        test_rmse_loss += rmseloss
        test_mae_loss += maeloss
        test_mre_loss += mreloss

    n = len(test_inputs)
    RMSE = test_rmse_loss / n
    MAE = test_mae_loss / n
    MRE = test_mre_loss / n

    print(f"RMSE: {RMSE}")
    print(f"MAE: {MAE}")
    print(f"MRE: {MRE}")


test_heterogeneous_model(heterogeneous_model, test_inputs)

RMSE: 0.006351334974169731
MAE: 0.0020139727275818586
MRE: 0.04707365483045578
